In [3]:
import random
import time

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data, datasets
SEED = 42

In [4]:
# Reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

I0508 18:22:38.505672 4649907648 tokenization_utils.py:1011] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/davina/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0508 18:22:38.892890 4649907648 filelock.py:274] Lock 112891729960 acquired on /Users/davina/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
I0508 18:22:38.893735 4649907648 file_utils.py:444] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /Users/davina/.cache/torch/transformers/tmp7gj3aem9
Downloading: 100%|██████████| 433/433 [00:00<00:00, 221kB/s]
I0508 18:22:39.274518 4649907648 file_utils.py:448] storing https://s3.amazonaws.com/models.hug

In [6]:
len(tokenizer.vocab)

30522